In [2]:
import os
import sys

import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

import tensorflow as tf
tf.random.set_seed(1)
np.random.seed(1)

In [3]:
df = pd.read_csv('/content/drive/MyDrive/bitamin/week6_Word Embedding & Transfer Learning/preview/data.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [4]:
docs = df['paragraph'].tolist()
categories = df['category'].tolist()

In [9]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)

def get_max_length(df):
  max_length = 0
  for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
      max_length = len(row.split(" "))
  return max_length

max_length = get_max_length(df)
print(max_length)

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

91


In [10]:
padded_docs[0]

array([162,  16, 163,   5, 164, 165,  56,  16, 166, 167,   1, 168,   9,
         6,  91,  25,  92,   2, 169,  40, 170, 171,  57,  26, 172,   3,
        12,  41,   2,  23,  58,   8, 173,  15,   4, 174,  10,  13,  59,
        93,  17,  12, 175, 176, 177, 178, 179,  94,  42,   1, 180,   2,
        95,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [5]:
# label encoding
def category_encode(category):
  if category == 'food':
    return [1, 0]
  else:
    return [0, 1]

labels = array([category_encode(category) for category in categories])
print("first document's label: ", labels[0])

first document's label:  [1 0]


In [6]:
# GloVe download
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename='glove.6B.zip')
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall()
zf.close()

In [13]:
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [14]:
# Model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(padded_docs, array(labels), epochs=5, verbose=1)

Epoch 1/5
1/1 [==============================] - 1s 606ms/step - loss: 0.7488 - accuracy: 0.3500
Epoch 2/5
1/1 [==============================] - 0s 14ms/step - loss: 0.3308 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 13ms/step - loss: 0.1589 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 7ms/step - loss: 0.0877 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 9ms/step - loss: 0.0477 - accuracy: 1.0000
